**Compile wrf chem: QUICK NOTE**
1. It is based on the NCAR wrf chem compile complete process and note on wrf chem compile by [this](http://sergioibarra.blogspot.in/2012/10/wrf-chem-compilacion-en-ubuntu-1204.html)
1. Basic environemtn testl, passes all the tests which checks libs such as, gcc, gfortran, gcc+gfortran, perl, csh.
1. Made sure the gcc and gfortran versions are matching, current case version is 4.8.2 for both gcc and gfortran
1. Basic library compile MPICh and NETCDF
1. For setting up PATH for NETCDF
```bash
export DIR=/home/ubuntu/wrf/libs/
export CC=gcc
export CXX=g++
export FC=gfortran
export FCFLAGS=-m64
export F77=gfortran
export FFLAGS=-m64
./configure --prefix=$DIR/netcdf --disable-dap \
     --disable-netcdf-4 --disable-shared
```
1. make (this command ends in error related to documentation, there is a patch for this error)
```bash
make install
export PATH=$DIR/netcdf/bin:$PATH
export NETCDF=$DIR/netcdf
```
1. setting up PATH for mpich and compileing
```bash
tar xzvf mpich-3.0.4.tar.gz     #or just .tar if no .gz present
cd mpich-3.0.4
./configure --prefix=$DIR/mpich
make
make install
export PATH=$DIR/mpich/bin:$PATH
```
1. Remianing requiered librries were installed by normal sudo apt-get command, libs requiered were of 
```bash
sudo apt-get install m4
sudo apt-get install libjasper-dev
sudo apt-get install libpng-dev
sudo apt-get install libnetcdf-dev
sudo apt-get install zlib1g-dev
sudo apt-get install build-essential
sudo apt-get install libcloog-ppl-dev
sudo apt-get install tcsh
sudo apt-get install libcloog-ppl0
```
1. Basic library check was undergone, which checks netcdf and mpich , this test was passed good.

**compiling WRF**

1. Edited the file sudo nano /etc/bash.bashrc, added following lines in it
```bash
export NETCDF=/home/ubuntu/wrf/libs/netcdf
export JASPERLIB=/usr/lib
export JASPERINC=/usr/include
export WRF_EM_CORE=1
export WRF_NMM_CORE=0
export WRF_CHEM=1
export WRF_KPP=0
export WRFIO_NCD_LARGE_FILE_SUPPORT=1
```
1. Inside the wrf chem folder in whcih the chem folder already copy pasted, run following commands
```bash
cd /home/hoopoe/wrfchem341/WRFV3_mpichA4/WRFV3/
./configure
```
1. The configure.wrf file has to be edited since the groftran and gcc used is of 4.8.2 version. The lines starting with has to be as below
```bash
FORMAT_FIXED    =       -ffixed-form -cpp
FORMAT_FREE     =       -ffree-form -ffree-line-length-none -cpp
```
1. Then execute the command, ./compile em_real and ./compile emi_conv. This completes WRF CHEM compilation
1. For compiling WPS, unzip the wps folder and run the command  ```./configure``` and choose serial based wps, it ends in creation of file configure.wps. 
1. Edit the file configure.wps and change the line starting with exactly as below
```bash
#### Architecture specific settings ####
COMPRESSION_LIBS    = -L/usr/lib -ljasper -lpng -lz
COMPRESSION_INC     = -I/usr/include
#
#   Settings for Linux x86_64, gfortran    (serial) 
#
#
COMPRESSION_LIBS    = -L/usr/lib -ljasper -lpng -lz
COMPRESSION_INC     = -I/usr/include
```
1. Chnage the line in wps due to gformartn version used 4.8.2 as, `CPP                 = /usr/bin/cpp -P -traditional`
1. Then execute the command, ./compile, and it will generate the exe required for WPS. 
1. Changed the lopcation of wrf compiled folder

**compiling PREP_CHEM_SRC**

1. compiling zlib1.7.2, inside the source directory, given follwoing command,
```bash
FC=gfortran CC=gcc ./configure --prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib
make
make install
```
1. compiling hdf5-1.8.4, inside the source directory, given following command,
```bash					
FC=gfortran CC=gcc CXX=gc++ ./configure  \
--prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/hdf \
--with-zlib=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib  \
--disable-shared \
--enable-fortran
make
make install
```
1. Then editing the prep_chme_src fconfigure file, ```include.mk.opt.gfortran```, with lines
```bash
NETCDF=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/netcdf
HDF5=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/hdf
HDF5_LIB=-L$(HDF5)/lib -lhdf5hl_fortran -lhdf5_fortran -lhdf5_hl -lhdf5 -L/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib/lib/ -lz
```
1. Made sure that the line contain wording for big endian memory as 
```bash
F_OPTS=  -Xpreprocessor -D$(CHEM) -O2 -fconvert=big-endian -frecord-marker=4
```
1. Then saved the configure file and given the command
```bash
cd /home/hoopoe/wrfchem341/WRFV3_mpichA3/PREP-CHEM-SRC-1.4/bin/build/
make OPT=gfortran.wrf CHEM=RACM
```
1. This ends in error saying 
```bash
H5PL.c:(.text+0xa7): undefined reference to `dlclose'
/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/hdf/			lib/libhdf5.a(H5PL.o): In function `H5PL_load':
H5PL.c:(.text+0x319): undefined reference to `dlsym'
H5PL.c:(.text+0x45c): undefined reference to `dlopen'
H5PL.c:(.text+0x475): undefined reference to `dlsym'
H5PL.c:(.text+0x5a4): undefined reference to `dlclose'
H5PL.c:(.text+0x629): undefined reference to `dlerror'
H5PL.c:(.text+0x83c): undefined reference to `dlclose'
collect2: error: ld returned 1 exit status
make: *** [../prep_chem_sources_RACM.exe] Error 1
```
1. So made another attmept of compiling HDF librar alone as follows, inside the source directory of compiling hdf5-1.8.4, given following command,
```bash					
FC=gfortran CC=gcc CXX=gc++ ./configure  \
--prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/hdfA2 \
--with-zlib=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib  \
--enable-shared --enable-hl\
--enable-fortran
make -j 3
make install
```
1. This new location of HDF compiled by enabling shared library and hl is linked in prep_chem_src soure and it made exe without any error.
1. But running the wrf chem with prep_chem_src_RACM ends in same end of file error. It is supected that the compilation command has to give make OPT=gfortran.wrf CHEM=RADM_WRF_FIM instead of make OPT=gfortran.wrf CHEM=RACM, for a proper run. While compileing with make OPT=gfortran.wrf CHEM=RADM_WRF_FIM, it ends in error of no hdf5.mod avilable in directory. It seems the disable -shared compilation does created the hdf5.mod, but ends in earlier error of undefined reference to `dlclose'.
1. So recompile the hdf with out any enable-fortran as done earlier. Inside the source directory of compiling hdf5-1.8.4, given following command,
```bash					
FC=gfortran CC=gcc CXX=gc++ ./configure  \
--prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/hdfA3 \
--with-zlib=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib  \
--enable-shared --enable-hl
make -j 3
make install
```
This also doesn't created the hdf5.mod file proving that the mod file creation is by shared library variable. 
1. One more trial with Inside the source directory of compiling hdf5-1.8.4, given following command,
```bash					
FC=gfortran CC=gcc CXX=gc++ ./configure  \
--prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/hdfAA4 \
--with-zlib=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib  \
--disable-shared --enable-fortran --enable-parallel
```
1. which ends in error of checking whether a simple MPI-IO program can be linked`... no configure: error: unable to link a simple MPI-IO application`
1. Based on [this](http://comments.gmane.org/gmane.comp.programming.hdf/5131) zlib also has to be compiled specific to enable-parallel
```bash
FC=gfortran CC=gcc ./configure --prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib_125A1 
          make
          make install
FC=gfortran CC=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/mpich/bin/mpicc CXX=gc++ ./configure  \
					--prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/hdfAA5  --enable-parallel\
					--with-zlib=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib_125A1  \
					--enable-shared --enable-hl --enable-fortran
```
1. So go back to the readme of the prep_chem_src, follow its version spcifications. to donwload hdf1.8.8 from http://www.hdfgroup.org/ftp/HDF5/releases/hdf5-1.8.8/src/hdf5-1.8.8.tar.gz and to downlaod zlib1.2.5 http://sourceforge.net/projects/libpng/files/zlib/1.2.5/zlib-1.2.5.tar.gz/download and to donwload the netcdf 4.1.1, wget ftp://ftp.unidata.ucar.edu/pub/netcdf/netcdf-4.1.1.tar.gz
```bash
FC=gfortran CC=gcc CXX=gc++ ./configure -prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA4/libs/hdfA1 --with-zlib=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib --enable-shared --enable-fortran
```
1. It is found that the prep_chme_src compilaiton as `make OPT=gfortran.wrf CHEM=RADM_WRF_FIM.` is problamatic due to netcdf. so go for with `netcdf 4.1.1` for wrf chem starting itsefl. But trying to compile `hdf1.8.8` with `zlib 1.2.5` ends in error of `make[2]: *** [h5diff.lo] Error 1`. So going with hdf1.8.14 and zlib 1.2.7.
1. The libraries `hdf1.8.14` and `zlib1.2.7` was compiled smoothly and then compiled PREP_CHEM_SRC using the gfortran.wrf and removing the `-lnetcdff` comments in it which ends the compilaiotn of prep_chem_src also.
1. Output from this compialtion is correctly read by wrf convert_emiss.exe system and thus ends the compilaiton of wrf chem and its auxillary compontnets.
```bash
FC=gfortran CC=gcc ./configure --prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA4/libs/zlib
          make
          make install
FC=gfortran CC=gcc CXX=gc++ ./configure -prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA4/libs/hdfA1 --with-zlib=/home/hoopoe/wrfchem341/WRFV3_mpichA4/libs/zlib --enable-shared --enable-fortran
wget http://www2.mmm.ucar.edu/wrf/src/WRFV3.4.1.TAR.gz
wget http://www2.mmm.ucar.edu/wrf/src/WPSV3.4.1.TAR.gz
wget http://www2.mmm.ucar.edu/wrf/src/WRFV3-Chem-3.4.1.TAR.gz
wget http://www2.mmm.ucar.edu/wrf/OnLineTutorial/compile_tutorial/tar_files/mpich-3.0.4.tar.gz
wget ftp://ftp.unidata.ucar.edu/pub/netcdf/netcdf-4.1.1.tar.gz
tar xzvf netcdf-4.1.1.tar.gz
cd 
wget http://zlib.net/fossils/zlib-1.2.7.tar.gz
wget http://www.hdfgroup.org/ftp/HDF5/current/src/hdf5-1.8.14.tar
gunzip WPSV3.4.1.TAR.gz
tar -xf WPSV3.4.1.TAR
```


** screen quick command**

1. To start a screen with name `screen -S wrfems`
1. To view running screen `screen -r`
1. to get into particular screen `screen -xr pid(of the screen)`
1. to get out of screen `CTRL+A+D`

** to install wrf ems**

```bash
chmod a+x ems_install.pl
perl ems_install.pl --install
```
1. tranfer file 
```bash
ssh -i /home/swl-sacon-dst/Desktop/gssa.pem ubuntu@ec2-54-255-173-125.ap-southeast-1.compute.amazonaws.com
ssh -i /home/swl-sacon-dst/Desktop/wrfa.pem ubuntu@ec2-54-224-94-144.compute-1.amazonaws.com
```

** Some codes **

1. for compiling
```bash
FC=gfortran CC=gcc ./configure --prefix=/home/ubuntu/wrf/libs/zlib
          make
          make install
tar xvf hdf5-1.8.14.tar
FC=gfortran CC=gcc CXX=gc++ ./configure -prefix=/home/ubuntu/wrf/libs/hdfA1 --with-zlib=/home/ubuntu/wrf/libs/zlib --disable-shared --enable-fortran
./configure -prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA4/libs/hdfA2 --with-zlib=/home/ubuntu/wrf/libs/zlib --enable-shared --enable-fortran
./configure --prefix=/usr/local --enable-shared --enable-hl --with-zlib=/home/ubuntu/wrf/libs/zlib --enable-fortran
make
sudo checkinstall
ftp://aftp.fsl.noaa.gov/divisions/taq/global_emissions/prep_chem_sources_v1.4_08aug2013.tar.gz
tar xzvf prep_chem_sources_v1.4_08aug2013.tar.gz
make OPT=gfortran.wrf CHEM=RADM_WRF_FIM
dpkg -L hdf5
sudo dpkg -r hdf5
#addressing the issue of hdf5 file missing in prep_chem_src.exe
./configure --prefix=/usr/local --enable-shared --enable-hl --with-zlib=/home/hoopoe/wrfchem341/WRFV3_mpichA4/libs/zlib127 --enable-fortran
make
sudo checkinstall
```
1. BIG overcome, ldconfig, solve the compile exe not found link 
```bash
make OPT=gfortran.wrf CHEM=RADM_WRF_FIM
#no missing file was reported in this compile, indicating for prep_chem_src, the locaiton of hdf5 is hardcoded in /usr/local/, so has to be compiled for /usr/local
```
1. For server
```bash
export DIR=/home/hoopoe/wrfchem341/WRFV3_mpichA4/libs
export PATH=$DIR/netcdf/bin:$PATH
export NETCDF=$DIR/netcdf
export PATH=$DIR/mpich/bin:$PATH
```